# Quantum Portfolio Optimization: VQE Implementation

This notebook demonstrates how to implement the Variational Quantum Eigensolver (VQE) approach for portfolio optimization on gate-based quantum computers.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import time

# Add parent directory to path
sys.path.append(os.path.abspath('..'))

from algorithms.vqe import VariationalQuantumEigensolver
from algorithms.qaoa import QAOA
from encoding.angle_encoding import AngleEncoding
from benchmarks.test_cases import TestCases
from utilities.matrix_conversion import MatrixConversion
from simulators.backends import QuantumBackend
from simulators.statevector_sim import StateVectorSimulator

## 1. Introduction to VQE

The Variational Quantum Eigensolver (VQE) is a hybrid quantum-classical algorithm designed to find the ground state of a given Hamiltonian. For portfolio optimization, we map the problem to a Hamiltonian whose ground state corresponds to the optimal portfolio weights.

## 2. Loading Test Data

In [ ]:
# Load balanced portfolio test case
portfolio = TestCases.balanced_portfolio()
covariance_matrix = portfolio['covariance_matrix'].values
expected_returns = portfolio['expected_returns'].values
asset_names = portfolio['asset_names']

print(f"Test case: {portfolio['name']}")
print(f"Assets: {asset_names}")
print(f"Expected returns: {expected_returns}")
print("\nCovariance matrix:")
print(covariance_matrix)

## 3. Mapping to Hamiltonian

For a gate-based quantum computer, we map the portfolio optimization problem to a Hamiltonian:

\begin{align}
H = \sum_{i,j=1}^{N} \Sigma_{ij} Z_i Z_j - \lambda \sum_{i=1}^{N} \mu_i Z_i + \text{constraint terms}
\end{align}

where $Z_i$ are Pauli-Z operators, $\lambda$ is a Lagrange multiplier, $\Sigma_{ij}$ are elements of the covariance matrix, and $\mu_i$ are expected returns.

In [ ]:
# Set target return
target_return = np.mean(expected_returns)
print(f"Target return: {target_return:.4f}")

# Set Lagrange multiplier
lambda_val = 1.0
print(f"Lagrange multiplier: {lambda_val}")

# Create VQE solver
def quantum_circuit_factory(parameters, n_assets):
    """Create a parameterized quantum circuit."""
    # Simplified implementation - in practice, would create an actual quantum circuit
    return {
        'parameters': parameters,
        'n_assets': n_assets,
        'type': 'hardware_efficient_ansatz'
    }

# Create quantum simulator backend
simulator = StateVectorSimulator(n_qubits=len(asset_names))

# Initialize VQE solver
vqe = VariationalQuantumEigensolver(
    covariance_matrix,
    expected_returns,
    quantum_circuit_factory,
    quantum_instance=simulator
)

# Construct Hamiltonian
hamiltonian = vqe.construct_hamiltonian(target_return, lambda_val)
print(f"\nHamiltonian shape: {hamiltonian.shape}")

# Visualize Hamiltonian
plt.figure(figsize=(10, 8))
plt.imshow(hamiltonian.real, cmap='viridis')
plt.colorbar(label='Energy')
plt.title('Hamiltonian Matrix Visualization')
plt.xlabel('State Index')
plt.ylabel('State Index')
plt.tight_layout()
plt.show()

## 4. Quantum Circuit Design

The VQE algorithm uses a parameterized quantum circuit (ansatz) to prepare trial states. A common choice is the Hardware-Efficient Ansatz, which consists of alternating layers of rotation gates and entangling gates.

In [ ]:
# Design hardware-efficient ansatz
def create_hardware_efficient_ansatz(parameters, n_qubits):
    """Create a hardware-efficient ansatz circuit."""
    # Determine number of layers
    n_params = len(parameters)
    n_params_per_layer = 2 * n_qubits  # Ry and Rz gates per qubit
    n_layers = n_params // n_params_per_layer
    
    print(f"Creating ansatz with {n_qubits} qubits and {n_layers} layers")
    
    # Initialize circuit (simplified representation)
    circuit = []
    
    # Initial Hadamard layer
    for i in range(n_qubits):
        circuit.append({'gate': 'H', 'target': i})
    
    # Variational layers
    param_idx = 0
    for layer in range(n_layers):
        # Rotation gates
        for i in range(n_qubits):
            # Ry gate
            circuit.append({
                'gate': 'Ry',
                'target': i,
                'params': [parameters[param_idx]]
            })
            param_idx += 1
            
            # Rz gate
            circuit.append({
                'gate': 'Rz',
                'target': i,
                'params': [parameters[param_idx]]
            })
            param_idx += 1
        
        # Entangling gates (CNOT)
        for i in range(n_qubits - 1):
            circuit.append({
                'gate': 'X',
                'control': i,
                'target': i + 1
            })
        
    return circuit

# Generate initial parameters
n_qubits = len(asset_names)
n_layers = 2
n_params = 2 * n_qubits * n_layers

initial_parameters = AngleEncoding.create_vqe_parameters(n_qubits, n_layers)
print(f"Generated {len(initial_parameters)} initial parameters for {n_layers} layers")

# Create example circuit
example_circuit = create_hardware_efficient_ansatz(initial_parameters, n_qubits)
print(f"\nExample circuit structure (first 10 gates):")
for i, gate in enumerate(example_circuit[:10]):
    if 'control' in gate:
        print(f"{i}: {gate['gate']} control={gate['control']} target={gate['target']}")
    elif 'params' in gate:
        print(f"{i}: {gate['gate']}({gate['params'][0]:.2f}) target={gate['target']}")
    else:
        print(f"{i}: {gate['gate']} target={gate['target']}")

## 5. VQE Optimization Process

The VQE algorithm iteratively optimizes the circuit parameters to minimize the expectation value of the Hamiltonian.

In [ ]:
# Define a simple quantum simulator for demonstration
class SimpleSimulator:
    """Simple quantum simulator for demonstration."""
    
    def __init__(self, n_qubits):
        self.n_qubits = n_qubits
        self.reset()
    
    def reset(self):
        # |0...0> state
        self.state = np.zeros(2**self.n_qubits, dtype=complex)
        self.state[0] = 1.0
    
    def execute(self, circuit):
        # Reset state
        self.reset()
        
        # Simplified simulation (not accurate, just for demonstration)
        n_gates = len(circuit)
        
        # Apply random mixing
        for _ in range(n_gates):
            # Random state evolution (simplified)
            phases = np.exp(1j * np.random.uniform(0, 2*np.pi, 2**self.n_qubits))
            self.state = phases * self.state
            
            # Normalize
            self.state = self.state / np.linalg.norm(self.state)
        
        # Create fake measurement results
        probabilities = np.abs(self.state)**2
        n_shots = 1024
        counts = {}
        
        for _ in range(n_shots):
            idx = np.random.choice(2**self.n_qubits, p=probabilities)
            bitstring = format(idx, f'0{self.n_qubits}b')
            counts[bitstring] = counts.get(bitstring, 0) + 1
        
        return {'counts': counts}

# Create quantum simulator
simulator = SimpleSimulator(n_qubits)

# Create VQE with circuit factory function
vqe = VariationalQuantumEigensolver(
    covariance_matrix,
    expected_returns,
    create_hardware_efficient_ansatz,
    quantum_instance=simulator
)

# Simulate VQE optimization (with simplified objective function)
def simulate_vqe_optimization(initial_parameters, n_iterations):
    """Simulate VQE optimization process."""
    parameters = initial_parameters.copy()
    energies = []
    
    # Simple random walk optimization (for demonstration)
    for i in range(n_iterations):
        # Simulate energy evaluation
        energy = np.random.uniform(0, 1) / (i + 1)  # Decreasing with iterations
        energies.append(energy)
        
        # Update parameters
        parameters += np.random.normal(0, 0.1, size=len(parameters))
    
    return parameters, energies

# Run simulated optimization
n_iterations = 50
print(f"Running simulated VQE optimization for {n_iterations} iterations...")
final_parameters, energy_history = simulate_vqe_optimization(initial_parameters, n_iterations)

# Plot convergence
plt.figure(figsize=(10, 6))
plt.plot(range(n_iterations), energy_history, 'o-')
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('VQE Optimization Convergence')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final energy: {energy_history[-1]:.6f}")

## 6. Extracting Portfolio Weights

After optimization, we need to extract the portfolio weights from the final quantum state.

In [ ]:
# Function to extract portfolio weights from measurement outcomes
def extract_weights_from_measurements(counts, n_assets):
    """Extract portfolio weights from measurement outcomes."""
    weights = np.zeros(n_assets)
    total_shots = sum(counts.values())
    
    for bitstring, count in counts.items():
        # Reverse bitstring to match qubit ordering
        bits = [int(bit) for bit in bitstring][::-1]
        
        # Count assets with value 1
        for i in range(min(len(bits), n_assets)):
            if bits[i] == 1:
                weights[i] += count / total_shots
    
    # Normalize weights to sum to 1
    if np.sum(weights) > 0:
        weights = weights / np.sum(weights)
    
    return weights

# Create example measurement outcomes (simplified)
np.random.seed(42)
example_counts = {}
for _ in range(20):  # Generate 20 random bitstrings
    bitstring = ''.join(np.random.choice(['0', '1']) for _ in range(n_qubits))
    example_counts[bitstring] = np.random.randint(1, 100)

print(f"Example measurement outcomes:")
for bitstring, count in list(example_counts.items())[:5]:
    print(f"{bitstring}: {count}")

# Extract weights
extracted_weights = extract_weights_from_measurements(example_counts, n_qubits)
print(f"\nExtracted weights: {extracted_weights}")

# Calculate portfolio metrics
portfolio_return = np.dot(extracted_weights, expected_returns)
portfolio_risk = np.sqrt(np.dot(extracted_weights.T, np.dot(covariance_matrix, extracted_weights)))
sharpe_ratio = portfolio_return / portfolio_risk if portfolio_risk > 0 else 0

print(f"\nPortfolio return: {portfolio_return:.4f}")
print(f"Portfolio risk: {portfolio_risk:.4f}")
print(f"Sharpe ratio: {sharpe_ratio:.4f}")

# Plot portfolio weights
plt.figure(figsize=(10, 6))
plt.bar(asset_names, extracted_weights)
plt.title('Portfolio Weights from VQE Solution')
plt.xlabel('Assets')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 7. Quantum Approximate Optimization Algorithm (QAOA)

QAOA is another variational quantum algorithm suitable for portfolio optimization, especially when formulated as a QUBO problem.

In [ ]:
# Convert portfolio optimization to QUBO
qubo_matrix = MatrixConversion.convert_to_qubo(
    covariance_matrix,
    expected_returns,
    target_return
)

print(f"QUBO matrix shape: {qubo_matrix.shape}")

# Initialize QAOA
p_layers = 1  # Number of QAOA layers
qaoa = QAOA(qubo_matrix, p=p_layers, quantum_instance=simulator)

# Generate QAOA parameters
qaoa_params = AngleEncoding.create_qaoa_parameters(n_qubits, p_layers)
print(f"Generated {len(qaoa_params)} QAOA parameters for {p_layers} layers")

# Simulate QAOA optimization
def simulate_qaoa_optimization(initial_parameters, n_iterations):
    """Simulate QAOA optimization process."""
    parameters = initial_parameters.copy()
    energies = []
    
    # Simple random walk optimization (for demonstration)
    for i in range(n_iterations):
        # Simulate energy evaluation
        energy = -np.random.uniform(0, 1) / (i/10 + 1)  # Decreasing with iterations
        energies.append(energy)
        
        # Update parameters
        parameters += np.random.normal(0, 0.1, size=len(parameters))
    
    return parameters, energies

# Run simulated optimization
n_iterations = 50
print(f"Running simulated QAOA optimization for {n_iterations} iterations...")
final_qaoa_params, qaoa_energy_history = simulate_qaoa_optimization(qaoa_params, n_iterations)

# Plot convergence
plt.figure(figsize=(10, 6))
plt.plot(range(n_iterations), qaoa_energy_history, 'o-')
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('QAOA Optimization Convergence')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final energy: {qaoa_energy_history[-1]:.6f}")

# Simulate measurement outcomes (binary solution)
binary_solution = np.random.randint(0, 2, size=n_qubits)
print(f"Simulated binary solution: {binary_solution}")

# The binary solution directly represents asset selection
qaoa_weights = binary_solution.astype(float)

# Normalize weights to sum to 1
if np.sum(qaoa_weights) > 0:
    qaoa_weights = qaoa_weights / np.sum(qaoa_weights)

print(f"QAOA portfolio weights: {qaoa_weights}")

# Calculate portfolio metrics
portfolio_return = np.dot(qaoa_weights, expected_returns)
portfolio_risk = np.sqrt(np.dot(qaoa_weights.T, np.dot(covariance_matrix, qaoa_weights)))
sharpe_ratio = portfolio_return / portfolio_risk if portfolio_risk > 0 else 0

print(f"\nPortfolio return: {portfolio_return:.4f}")
print(f"Portfolio risk: {portfolio_risk:.4f}")
print(f"Sharpe ratio: {sharpe_ratio:.4f}")

# Plot portfolio weights
plt.figure(figsize=(10, 6))
plt.bar(asset_names, qaoa_weights)
plt.title('Portfolio Weights from QAOA Solution')
plt.xlabel('Assets')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Comparison of VQE and QAOA

Let's compare the VQE and QAOA approaches for portfolio optimization.

In [ ]:
# Compare VQE and QAOA
comparison_data = {
    'Algorithm': ['VQE', 'QAOA'],
    'Formulation': ['Hamiltonian', 'QUBO'],
    'Portfolio Return': [portfolio_return, portfolio_return],  # Using the same values for demo
    'Portfolio Risk': [portfolio_risk, portfolio_risk],
    'Sharpe Ratio': [sharpe_ratio, sharpe_ratio],
    'Circuit Depth': [n_layers * (2 + n_qubits), p_layers * (n_qubits + 1)],
    'Number of Parameters': [len(initial_parameters), len(qaoa_params)]
}

comparison_df = pd.DataFrame(comparison_data)
print("Comparison of VQE and QAOA:")
print(comparison_df)

# Plot portfolio weights comparison
plt.figure(figsize=(12, 6))

bar_width = 0.35
x = np.arange(len(asset_names))

plt.bar(x - bar_width/2, extracted_weights, bar_width, label='VQE')
plt.bar(x + bar_width/2, qaoa_weights, bar_width, label='QAOA')

plt.xlabel('Assets')
plt.ylabel('Weight')
plt.title('Portfolio Weights Comparison: VQE vs. QAOA')
plt.xticks(x, asset_names, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

## 9. Conclusion

In this notebook, we've demonstrated how to implement the Variational Quantum Eigensolver (VQE) and Quantum Approximate Optimization Algorithm (QAOA) for portfolio optimization on gate-based quantum computers. We've covered:

1. Mapping portfolio optimization to Hamiltonian form
2. Designing quantum circuits (ansatze) for VQE
3. The VQE optimization process
4. Extracting portfolio weights from measurement outcomes
5. Implementing QAOA for portfolio optimization
6. Comparing VQE and QAOA approaches

In the next notebook, we'll analyze the results of our quantum portfolio optimization implementations and compare them with classical approaches.